<a href="https://colab.research.google.com/github/jnsto/DEL_5.0_MCMC/blob/main/mcmc_hubble_tension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEL v4.1 MCMC - **概念実証 (Proof of Concept)**

**11点データで DEL vs ΛCDM を比較！**

## **重要な注意**

**これは概念実証（Proof of Concept）です**

- **データ**: わずか11点（実際のCMB解析は数千点）
- **Likelihood**: 簡易的なGaussian近似
- **ΔAIC = +6.7** は別の完全解析（Planck + BAO + SNe）での結果（v4.0 p.8）

**このノートブックの目的**:
- DEL理論の基本動作確認
- MCMCフレームワークのテスト
- パラメータ推定の概念実証

**完全な解析は別途実施中**

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy

In [ ]:
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

In [ ]:
# --- データ (11点) ---
z_obs = np.array([0.0, 0.07, 0.1, 0.2, 0.35, 0.57, 0.44, 0.6, 0.73, 1.5, 2.34])
H_obs = np.array([73.0, 78.2, 81.2, 88.8, 92.8, 97.3, 95.0, 98.8, 105.0, 140.0, 180.0])
err = np.array([1.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0, 15.0])

In [ ]:
# --- モデル関数 ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    ratio = np.clip(a / at, 1e-10, 1e10)
    term1 = (1 - S) * ratio**(-4)
    term2 = S * (1 + epsilon * (a - at))
    return np.clip(term1 + term2, 0, 10)

def H_z_DEL(z, H0, Om, at, gamma, epsilon):
    a = np.clip(1 / (1 + z), 1e-10, 1)
    Or = 8e-5
    rho_L = rho_lambda_DEL(a, at, gamma, epsilon)
    OL = 1 - Om - Or
    inside = Or * a**(-4) + Om * a**(-3) + OL * rho_L
    return H0 * np.sqrt(np.maximum(inside, 1e-10))

def H_z_LCDM(z, H0, Om):
    a = np.clip(1 / (1 + z), 1e-10, 1)
    Or = 8e-5
    OL = 1 - Om - Or
    inside = Or * a**(-4) + Om * a**(-3) + OL
    return H0 * np.sqrt(np.maximum(inside, 1e-10))

In [ ]:
# --- 尤度関数 ---
def log_likelihood_DEL(params):
    H0, Om, at, gamma, epsilon = params
    if not (67 < H0 < 74 and 0.28 < Om < 0.32 and 
            2.5e-4 < at < 4e-4 and 8 < gamma < 12 and 
            0.010 < epsilon < 0.025):
        return -1e10
    try:
        model = H_z_DEL(z_obs, H0, Om, at, gamma, epsilon)
        if np.any(~np.isfinite(model)):
            return -1e10
        chi2 = np.sum(((H_obs - model) / err) ** 2)
        return -0.5 * chi2
    except:
        return -1e10

def log_likelihood_LCDM(params):
    H0, Om = params
    if not (67 < H0 < 74 and 0.28 < Om < 0.32):
        return -1e10
    try:
        model = H_z_LCDM(z_obs, H0, Om)
        if np.any(~np.isfinite(model)):
            return -1e10
        chi2 = np.sum(((H_obs - model) / err) ** 2)
        return -0.5 * chi2
    except:
        return -1e10

In [ ]:
# --- 初期値生成関数 ---
def initialize_walkers(center, spread, bounds, nwalkers):
    ndim = len(center)
    p0 = []
    attempts = 0
    max_attempts = nwalkers * 100
    
    while len(p0) < nwalkers and attempts < max_attempts:
        sample = np.random.normal(center, spread)
        if all(bounds[i][0] < sample[i] < bounds[i][1] for i in range(ndim)):
            p0.append(sample)
        attempts += 1
    
    if len(p0) < nwalkers:
        print(f"Warning: Only initialized {len(p0)}/{nwalkers} walkers")
        while len(p0) < nwalkers:
            sample = [np.random.uniform(bounds[i][0], bounds[i][1]) for i in range(ndim)]
            p0.append(sample)
    
    return np.array(p0)

In [ ]:
# --- DEL MCMC ---
print("="*60)
print("Running DEL MCMC...")
print("="*60)

ndim_DEL = 5
nwalkers = 32

bounds_DEL = [
    (67, 74),
    (0.28, 0.32),
    (2.5e-4, 4e-4),
    (8, 12),
    (0.010, 0.025)
]

p0_DEL = initialize_walkers(
    [70.0, 0.30, 3.1e-4, 10.0, 0.017],
    [0.5, 0.005, 0.1e-4, 0.3, 0.001],
    bounds_DEL,
    nwalkers
)

print(f"Initialized {len(p0_DEL)} walkers")

sampler_DEL = emcee.EnsembleSampler(nwalkers, ndim_DEL, log_likelihood_DEL)
sampler_DEL.run_mcmc(p0_DEL, 5000, progress=True)

# 収束診断
try:
    tau = sampler_DEL.get_autocorr_time(quiet=True)
    print(f"\nAutocorrelation time: {tau}")
    print(f"Mean tau: {np.mean(tau):.1f}")
except:
    print("Could not compute autocorrelation time")

samples_DEL = sampler_DEL.get_chain(discard=1500, thin=15, flat=True)
log_prob_DEL = sampler_DEL.get_log_prob(discard=1500, flat=True)
chi2_DEL = -2 * np.max(log_prob_DEL)

print(f"\nDEL Results:")
print(f"  Best χ² = {chi2_DEL:.2f}")
print(f"  Samples: {len(samples_DEL)}")

In [ ]:
# --- ΛCDM MCMC ---
print("\n" + "="*60)
print("Running ΛCDM MCMC...")
print("="*60)

ndim_LCDM = 2

bounds_LCDM = [
    (67, 74),
    (0.28, 0.32)
]

p0_LCDM = initialize_walkers(
    [70.0, 0.30],
    [0.5, 0.005],
    bounds_LCDM,
    nwalkers
)

sampler_LCDM = emcee.EnsembleSampler(nwalkers, ndim_LCDM, log_likelihood_LCDM)
sampler_LCDM.run_mcmc(p0_LCDM, 3000, progress=True)

samples_LCDM = sampler_LCDM.get_chain(discard=1000, flat=True)
log_prob_LCDM = sampler_LCDM.get_log_prob(discard=1000, flat=True)
chi2_LCDM = -2 * np.max(log_prob_LCDM)

print(f"\nΛCDM Results:")
print(f"  Best χ² = {chi2_LCDM:.2f}")
print(f"  Samples: {len(samples_LCDM)}")

In [ ]:
# --- ΔAIC 計算 ---
k_DEL = 5
k_LCDM = 2
delta_k = k_DEL - k_LCDM
delta_chi2 = chi2_LCDM - chi2_DEL
delta_AIC = delta_chi2 - 2 * delta_k

print("\n" + "="*60)
print("FINAL RESULTS (11-point Proof of Concept)")
print("="*60)
print(f"χ²_DEL   = {chi2_DEL:.2f}  (k = {k_DEL})")
print(f"χ²_ΛCDM  = {chi2_LCDM:.2f}  (k = {k_LCDM})")
print(f"Δχ²      = {delta_chi2:+.2f}")
print(f"Δk       = {delta_k}")
print(f"ΔAIC     = {delta_AIC:+.2f}")
print("="*60)

if delta_AIC > 0:
    print(f"DEL is preferred (ΔAIC = +{delta_AIC:.1f})")
    if delta_AIC > 10:
        print("  → Very strong evidence")
    elif delta_AIC > 5:
        print("  → Strong evidence")
    elif delta_AIC > 2:
        print("  → Moderate evidence")
    else:
        print("  → Weak evidence")
else:
    print(f"ΛCDM is preferred (ΔAIC = {delta_AIC:.1f})")

print("\nNote: This is a proof of concept with 11 data points.")
print("    Full analysis (Planck + BAO + SNe) reported ΔAIC = +6.7 (v4.0)")

In [ ]:
# --- コーナープロット ---
fig = corner.corner(
    samples_DEL,
    labels=['H₀', 'Ωₘ', 'aₜ', 'γ', 'ε'],
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_fmt='.4f'
)
plt.suptitle('DEL Parameter Constraints (11-point PoC)', y=1.02)
plt.savefig('del_corner.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nAnalysis complete! Corner plot saved as 'del_corner.png'")